# <center>Stock Price Fluctation</center>

# Phân tích yêu cầu của công ty
## Yêu cầu của công ty
Hãy sử dụng dữ liệu được cung cấp trong email này (dữ liệu giá và khối lượng của một vài mã cổ phiếu) để xây dựng một số mô hình dự đoán biến động giá cổ phiếu.
**Biến động giá cổ phiếu = Giá cổ phiếu N (phút hoặc giờ hoặc ngày) sau - Giá cổ phiếu hiện tại.**
Công ty cung cấp 4 file dữ liệu từ các công ty FPT, MSN, PNJ và VIC bao gồm các trường
- Open: Giá mở bán cổ phiếu
- High: Giá cao nhất của cổ phiếu trong khoảng thời gian đó.
- Low: Giá thấp nhất trong khoảng thời gian đó.
- Close: Giá đóng
- Volume: Khối lượng giao dịch
# Phân tích
## Xác định target value
$StockPriceFluctuation = Close_{t+1} - Close_{t}$

# Import library

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# read data

In [ ]:

def load_and_process_stock_data(file_paths, N):
    dataframes = []
    
    for file_path in file_paths:
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Ensure the dataframe has the required columns
        required_columns = ['Ticker', 'Date/Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']
        if not all(col in df.columns for col in required_columns):
            print(f"Warning: {file_path} is missing some required columns. Skipping this file.")
            continue
        
        # Convert Date/Time to datetime
        df['Date/Time'] = pd.to_datetime(df['Date/Time'], format='%m/%d/%Y %H:%M')
        
        # Sort by Date/Time in descending order (newest first)
        df = df.sort_values('Date/Time', ascending=False).reset_index(drop=True)
        
        # Calculate price fluctuation
        df['StockFluctuation'] = df['Close'] - df['Close'].shift(-N)
        
        # Drop rows with NaN values resulting from the shift operation
        df = df.dropna()
        
        dataframes.append(df)
    
    # Combine all dataframes
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    return combined_df

# List of file paths (adjust these to match your actual file paths)
file_paths = [
    'data/FPT.csv',
    'data/MSN.csv',
    'data/PNJ.csv',
    'data/VIC.csv'
]

# Set N for the number of periods to look ahead (e.g., 1 for next minute, 5 for 5 minutes ahead, etc.)
N = 1

# Process the data
result_df = load_and_process_stock_data(file_paths, N)

# Display the first few rows and info of the resulting dataframe
print(result_df.head())
print(result_df.info())

# Optional: Save the processed data to a new CSV file
result_df.to_csv('processed_stock_data.csv', index=False)
print("Processed data saved to 'processed_stock_data.csv'")

In [ ]:
# Optional: Group by Ticker and display summary statistics
summary_stats = result_df.groupby('Ticker').agg({
    'Open': 'mean',
    'High': 'max',
    'Low': 'min',
    'Close': 'mean',
    'Volume': 'sum',
    'StockFluctuation': ['mean', 'std']
})
print("\nSummary Statistics by Ticker:")
print(summary_stats)

# Data preprocessing


## Scaling data

In [15]:
# Initialize scalers for features and target
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# Define feature columns to scale
feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume']

# Apply scaling for each company
scaled_dfs = []
for ticker, group in result_df.groupby('Ticker'):
    # Scale features
    group[feature_columns] = feature_scaler.fit_transform(group[feature_columns])
    # Scale target (StockFluctuation)
    group['StockFluctuation'] = target_scaler.fit_transform(group[['StockFluctuation']])
    scaled_dfs.append(group)

# Combine back the scaled data
scaled_combined_df = pd.concat(scaled_dfs, axis=0)


## Create Time Series Sequences for LSTM

In [ ]:
def create_sequences(data, feature_columns, target_column, lookback):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[feature_columns].iloc[i:i+lookback].values)
        y.append(data[target_column].iloc[i+lookback])
    return np.array(X), np.array(y)

X_list, y_list = [], []
lookback_period = 30

for ticker, group in scaled_combined_df.groupby('Ticker'):
    X, y = create_sequences(group, feature_columns, 'StockFluctuation', lookback_period)
    X_list.append(X)
    y_list.append(y)

# Concatenate all the sequences from different companies
X_combined = np.concatenate(X_list, axis=0)
y_combined = np.concatenate(y_list, axis=0)


## Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, shuffle=False)

# Build LSTM model


In [ ]:

model = Sequential()

# Add LSTM layers
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))

# Output layer to predict stock fluctuation
model.add(Dense(units=1))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
model.save('keras_model.h5')

## Evaluate

In [ ]:
y_pred = model.predict(X_test)
y_pred_rescaled = target_scaler.inverse_transform(y_pred)
y_test_rescaled = target_scaler.inverse_transform(y_test.reshape(-1, 1))

### MAE

In [ ]:
mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
mae_percentage = (mae / np.mean(y_test)) * 100
print("Mean absolute error on test set: {:.2f}%".format(mae_percentage))

### R2 score

In [ ]:
r2 = r2_score(y_test_rescaled, y_pred_rescaled)
print("R2 score:", r2)